In [ ]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import seaborn as sns
sns.set_context('talk')

In [ ]:
# # This is for creating scatter plots of true vs predicted

# for c in ndf:
#     if c in ['datetime']:
#         continue
#     if c not in ['r0']:
#         continue

#     fig, ax = plt.subplots(1, 1, figsize=(20, 10))

#     wins = pd.DataFrame(index=range(data.size))
#     data = ndf[c]

#     # Create interpolation predictions and average the results per row
#     for i in range(t, data.shape[0]-m-t):
#         pred = predict(data, i, m, t, method)
#         wins[i] = np.nan
#         wins[i][i:i+m] = pred
#     mean = wins.mean(axis=1)

#     ax.scatter(x=mean.index, y=mean.values, s=10, color='red' , alpha=1, label='Predicted')
#     ax.scatter(x=data.index, y=data.values, s=10, color='blue', alpha=1, label='True')
#     ax.set_title(f'Predicted vs True for {c}\n{time}\nm={m}, t={t}, method={method}')

#     ax.legend()
#     ax.set_xlabel('Time')
#     ax.set_ylabel(c)

#     # Reset xticks to be datetimes
#     ticks = ax.get_xticks()
#     ticks = [0.0] + list(ticks[1:-2]) + [ndf.index[-1]]
#     ax.set_xticklabels(ndf.loc[ticks].datetime, rotation=45)

#     plt.tight_layout()
#     plt.savefig(f'local/plots/subs/{c}.{method}.{block}.png')


In [ ]:
# This code creates the error plots
# `data` is the dataframe output of window_interpolate.py
mplier = 5
def draw_error_plots(data, output, center='mean', error='std', subtitle=''):
    m = np.unique(data.index.get_level_values('m'))
    t = np.unique(data.index.get_level_values('t'))
    k = np.unique(data.index.get_level_values('kind'))
    ncols = len(t)
# #     win = 10
    for c in np.unique(data.columns.get_level_values(0)):
        fig, axes = plt.subplots(ncols=ncols, nrows=1, sharey=True, figsize=(10*ncols, 10))
        for i, _t in enumerate(t):
            ax = axes[i]
            ax.set_title(f'{c} | test={_t*mplier} (mins)\n{subtitle}')
            ax.set_xlabel('train (mins)')
            ax.set_xticks(m*mplier)
            ax.set_ylabel('Error')
            for _k in k:
                mean = data[c, center][:, _t, _k]
                yerr = data[c, error ][:, _t, _k]
#                 ax.errorbar(x=m*mplier, y=mean, yerr=yerr, fmt='-o', label=_k)
                ax.plot(m*mplier, mean, label=_k)

            ax.legend()
            plt.tight_layout()
            plt.savefig(f'{output}.{c}.png')


In [ ]:
# Manually set these; file is the output of window_interpolate.py
file   = '/Users/urebbapr/research/opticalcomm/data/interp/temperature.h5'

# this dict is just for creating the filename and subtitles to differentiate the different plots
# blocks = {
#     'spring': '04-01',
#     'summer': '07-01',
#     'fall'  : '10-01',
#     'winter': '12-01'
# }

blocks = {
    'results':'2019-01-01'
}
subtitle = '12pm to 12am PST'
for key, date in blocks.items():
    data = pd.read_hdf(file, key)
#     draw_error_plots(data, f'local/plots/subs/percent/{season}.{date}', subtitle=f'{date} from {subtitle}')
    draw_error_plots(data, f'{key}.{date}')

In [ ]:
t = 5
method = 'linear'

In [ ]:
def predict(data, i, m, t, kind):
    testing  = data.iloc[i:i+m]

    t1 = data.iloc[i-t : i]
    t2 = data.iloc[i+m : i+m+t]

    try:
        assert t1.isnull().mean() < .25
        assert t2.isnull().mean() < .25

        training = pd.concat([t1, t2])

        interp  = interp1d(x=training.index, y=training.values, kind=kind)
        predict = interp  (x=testing.index)

        diff = testing - predict

        return predict
    except Exception as e:
        print(e)

In [ ]:
## SETUP
# import data
file = '/Users/urebbapr/research/opticalcomm/data/v3/data.h5'
df = pd.read_hdf(file, 'merged')

In [ ]:
# ndf is typically my 'new' df / secondary df; in this case, date subset
ndf = df.loc[('2020-01-01 00:00' <= df.index) & (df.index < '2020-01-31 00:00')]
# ndf = ndf.reset_index() # Index needs to be integers

# How to do interpolation predictions

In [ ]:
#         fig, axes = plt.subplots(ncols=ncols, nrows=nrows, sharey=True, figsize=(10*ncols, 10*nrows))



#                 win = data[c, 'windows'][m[0], _t, _k] # a hard-coded 5 here


#                         maxv = data[c, center][_m].max()+data[c, error][_m].max()
#                         minv = data[c, center][_m].max()+data[c, error][_m].min()

# ax.set_ylim(
                        #     ymin=min([0, data[c, center][_m].min()-data[c, error][_m].max()]),
                        #     ymax=data[c, center][_m].max()+data[c, error][_m].max()
                        # )

#                     ax.set_ylim(
    #                         ymin=minv,
    #                         ymax=maxv
    #                     )
                        # ax.set_ylim(ymin=data[c, 'std'][_m].min(), ymax=data[c, 'std'][_m].max())
        
    #                     ax.hlines(0, *ax.get_xlim(), color='green', alpha=.5)
#                 for _m in m:
#                     color = 'blue'
#                     if data[c, 'windows'][_m, _t, _k] != win:
#                         color = 'orange'
#                     mean = data[c, center][_m, _t, _k]
#                     yerr = data[c, error ][_m, _t, _k]
#                     ax.errorbar(x=_m, y=mean, yerr=yerr, fmt='-o', color=color, ecolor='red', label=_k)
